In [103]:
import psycopg2
from tabulate import tabulate

In [104]:
con = psycopg2.connect(host="localhost",database="postgres",user="postgres",password="Postgres",port=5432)

In [40]:
# For isolation: SERIALIZABLE
con.set_isolation_level(3)
# For atomicity
con.autocommit = False

In [41]:
def xecuteproject(query, tablename):
    cur.execute(query)
    res = cur.fetchall()
    if tablename == 'Product':
        print('--------------PRODUCT TABLE-------------------')
        print(tabulate(res, headers=['prodid', 'deptid', 'quantity'], tablefmt='psql'))

    if tablename == 'Depot':
        print('--------------DEPOT TABLE-------------------')
        print(tabulate(res, headers=['deptid', 'addr','volume'], tablefmt='psql'))

    if tablename == 'Stock':
        print('--------------STOCK TABLE-------------------')
        print(tabulate(res, headers=['prodid', 'deptid', 'quantity'], tablefmt='psql'))

In [108]:
cur = con.cursor()
query1 = 'Select * from productassign5'
query2 = 'Select * from depot_assign5'
query3 = 'Select * from stock_assign5'

xecuteproject(query1, 'Stock')
xecuteproject(query2, 'Product')
xecuteproject(query3, 'Depot')    

--------------STOCK TABLE-------------------
+---------+--------+----------+
| depid   | addr   |   volume |
|---------+--------+----------|
| p2      | tv     |    250   |
| p3      | vcr    |     80   |
| p1      | tape   |      2.5 |
+---------+--------+----------+
--------------PRODUCT TABLE-------------------
+-------------+----------+------------+
| productid   | deptid   |   quantity |
|-------------+----------+------------|
| d2          | Syracuse |       6000 |
| d4          | New York |       2000 |
| d1          | New York |       9000 |
+-------------+----------+------------+
--------------DEPOT TABLE-------------------
+-------------+----------+------------+
| productid   | deptid   |   quantity |
|-------------+----------+------------|
| p3          | d1       |       2000 |
| p2          | d1       |       -400 |
| p1          | d1       |       1000 |
| p3          | d4       |       2000 |
| p2          | d4       |       1500 |
| p2          | d2       |       2000 |

In [109]:
# The product p1 is deleted from Product and Stock.
try:
    query4 = "Delete from productassign5 where productid='p1'"
    print('---Delete from productassign5 where productid------')
    cur.execute(query4)
    cur.execute(query1)
    res = cur.fetchall()
    print('--------------PRODUCT TABLE-------------------')
    print(tabulate(res, headers=['productid', 'deptid', 'quantity'], tablefmt='psql'))
    
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    print("Either it is Primary key violation or Please check if there is no row with name p1")
    con.rollback()

finally:
    if con:
        con.commit()

---Delete from productassign5 where productid------
--------------PRODUCT TABLE-------------------
+-------------+----------+------------+
| productid   | deptid   |   quantity |
|-------------+----------+------------|
| p2          | tv       |        250 |
| p3          | vcr      |         80 |
+-------------+----------+------------+


In [110]:
# Product deleted from Stock
try:
    query5 = "Delete from stock_assign5 where prodid='p1'"
    print('Delete the product p1 from the STOCK TABLE--')
    cur.execute(query5)
    cur.execute(query3)
    res = cur.fetchall()
    print(tabulate(res, headers=['productid', 'deptid', 'quantity'], tablefmt='psql'))
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    print("Either it is Primary key violation or Please check if there is no row with name p1")
    con.rollback()

finally:
    if con:
        con.commit()

Delete the data from the STOCK TABLE--
+-------------+----------+------------+
| productid   | deptid   |   quantity |
|-------------+----------+------------|
| p3          | d1       |       2000 |
| p2          | d1       |       -400 |
| p3          | d4       |       2000 |
| p2          | d4       |       1500 |
| p2          | d2       |       2000 |
+-------------+----------+------------+


In [111]:
# Query2 - Depot d1 is deleted from Depot.
try:
    query6 = "Delete from depot_assign5 where depid='d1'"
    cur.execute(query6)
    print('-------Depot TABLE---------------------------')
    cur.execute(query2)
    res = cur.fetchall()
    print(tabulate(res, headers=['depid', 'addr', 'volume'], tablefmt='psql'))
# Query7 - Depot d1 is deleted  Stock table
    print("Deleting data from the Stock d1 and delete from Depot and Stock. ")
    query7 = "Delete from stock_assign5 where deptid = 'd1'"
    cur.execute(query7)
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    print("Either it is Primary key violation or Please check if there is no row with name p1")
    con.rollback()

finally:
    if con:
        con.commit()


-------Depot TABLE---------------------------
+---------+----------+----------+
| depid   | addr     |   volume |
|---------+----------+----------|
| d2      | Syracuse |     6000 |
| d4      | New York |     2000 |
+---------+----------+----------+
Deleting data from the Stock d1 and delete from Depot and Stock. 


In [112]:
# Inserting rows with p1
try:
    cur.execute(query3)
    res = cur.fetchall()
    print('----DELETED D1 from---Stock TABLE---------------------------')
    print(tabulate(res, headers=['deptid', 'addr', 'volume'], tablefmt='psql'))
    # Check for product p1 exists and if not insert rows
    print(query1)
    cur.execute(query1)
    res = cur.fetchall()
    print('--------------PRODUCT TABLE-------------------')
    print(tabulate(res, headers=['productid', 'pname', 'price'], tablefmt='psql'))
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    print("Either it is Primary key violation or Please check if there is no row with name p1")
    con.rollback()

finally:
    if con:
        con.commit()


----DELETED D1 from---Stock TABLE---------------------------
+----------+--------+----------+
| deptid   | addr   |   volume |
|----------+--------+----------|
| p3       | d4     |     2000 |
| p2       | d4     |     1500 |
| p2       | d2     |     2000 |
+----------+--------+----------+
Select * from productassign5
--------------PRODUCT TABLE-------------------
+-------------+---------+---------+
| productid   | pname   |   price |
|-------------+---------+---------|
| p2          | tv      |     250 |
| p3          | vcr     |      80 |
+-------------+---------+---------+


#We add a product (p100, cd, 5) in Product and (p100, d2, 50) in Stock

In [113]:
try:
    insertquery = "insert into productassign5(productid,pname,price) values('p1','tape',2.5)"
    cur.execute(insertquery)
    cur.execute(query1)
    res = cur.fetchall()
    print('--------------After inserting product p1 into PRODUCT TABLE-------------------')
    print(tabulate(res, headers=['productid', 'pname', 'price'], tablefmt='psql'))
    query8 = "Update productassign5 SET productid = 'pp1' where productid='p1'"
    cur.execute(query8)
    cur.execute(query1)
    #print('The product p1 changes its name to pp1 in Product and Stock.')
    res = cur.fetchall()
    
except(Exception, psycopg2.DatabaseError):
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    print("Either it is Primary key violation or Please check if there is no row with name p1")
    con.rollback()

finally:
    if con:
        con.commit()
      

--------------After inserting product p1 into PRODUCT TABLE-------------------
+-------------+---------+---------+
| productid   | pname   |   price |
|-------------+---------+---------|
| p2          | tv      |   250   |
| p3          | vcr     |    80   |
| p1          | tape    |     2.5 |
+-------------+---------+---------+
The product p1 changes its name to pp1 in Product and Stock.


In [114]:
print('--------------PRODUCT TABLE-------------------')
print(tabulate(res, headers=['productid', 'pname', 'price'], tablefmt='psql'))
print('product p1 changed to pp1 in Product table')

--------------PRODUCT TABLE-------------------
+-------------+---------+---------+
| productid   | pname   |   price |
|-------------+---------+---------|
| p2          | tv      |   250   |
| p3          | vcr     |    80   |
| pp1         | tape    |     2.5 |
+-------------+---------+---------+
product p1 changed to pp1 in Product table


In [116]:
# Query9--The product p1 changes its name to pp1 in Stock.
try:
    print('------------STOCK ASSIGN5-------------------------')
    insertquery = "insert into stock_assign5(prodid,deptid,quantity)values('p1','tape',2.5)"
    query9 = "Update stock_assign5 SET prodid='pp1' where prodid='p1'"
    cur.execute(insertquery)
    cur.execute(query9)
    cur.execute(query3)
    res = cur.fetchall()
    print(tabulate(res, headers=['productid', 'deptid', 'quantity'], tablefmt='psql'))
    
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Transactions could not be completed so database will be rolled back before start of transactions")
    print("Either it is Primary key violation or Please check if there is no row with name p1")
    con.rollback()

finally:
    if con:
        con.commit()

------------STOCK ASSIGN5-------------------------
+-------------+----------+------------+
| productid   | deptid   |   quantity |
|-------------+----------+------------|
| pp1         | tape     |        2.5 |
| p3          | d4       |     2000   |
| p2          | d4       |     1500   |
| p2          | d2       |     2000   |
+-------------+----------+------------+


In [78]:
# Query10 -- Depot d1 changes its name to dd1 in Stock 00 THis gave Serizalization ERROR


--------------Verify if d1 is in Stock Table-------------
+----------+----------+------------+
| prodid   | deptid   |   quantity |
|----------+----------+------------|
| p3       | d4       |       2000 |
| p2       | d4       |       1500 |
| p2       | d2       |       2000 |
| p3       | d1       |       2000 |
| p2       | d1       |       -400 |
| pp1      | d1       |       1000 |
| pp1      | d2       |       -100 |
| pp1      | d4       |       1200 |
+----------+----------+------------+


In [126]:
try:
    print('--------------Verify if d1 is in Depot Table-------------')
    query = "Select * from depot_assign5"
    #delquery = "Delete from stock_assign5 where deptid = 'tape'"
    # Testing Serialization and Handling it
    cur.execute(delquery)
    cur.execute(query)
    res2 = cur.fetchall()
    print("---Current Depot table")
    print(tabulate(res2, headers=['depid', 'addr', 'volume'], tablefmt='psql'))
    print("Insert d1 row into the table")
    insertquery = "insert into depot_assign5(depid,addr,volume)values('d1','New York',9000)"
    cur.execute(insertquery)
    res = cur.fetchall()
    print(tabulate(res, headers=['depid', 'addr', 'volume'], tablefmt='psql'))
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Either it is Primary key violation or Please check if there is no row with name ")
    con.rollback()

finally:
    if con:
        con.commit()


--------------Verify if d1 is in Depot Table-------------
---Current Depot table
+---------+----------+----------+
| depid   | addr     |   volume |
|---------+----------+----------|
| d2      | Syracuse |     6000 |
| d4      | New York |     2000 |
| d1      | New York |     9000 |
+---------+----------+----------+
Insert d1 row into the table
duplicate key value violates unique constraint "depot_assign5_pkey"
DETAIL:  Key (depid)=(d1) already exists.

Either it is Primary key violation or Please check if there is no row with name p1


In [130]:
# Issue of Concurrent execution
try:
    print(tabulate(res, headers=['depid', 'addr', 'volume'], tablefmt='psql'))
    query10_1 = "Update depot_assign5 SET depid = 'dd1' where depid='d1'"
    print('---Depot d1 changes its name to dd1 in Depot------')
    cur.execute(query10_1)
    cur.execute(query2)
    res = cur.fetchall()
    #print(tabulate(res, headers=['depid', 'addr', 'volume'], tablefmt='psql'))
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Either it is Primary key violation or Please check if there is no row with name ")
    con.rollback()

finally:
    if con:
        con.commit()

+---------+----------+----------+
| depid   | addr     |   volume |
|---------+----------+----------|
| d2      | Syracuse |     6000 |
| d4      | New York |     2000 |
| dd1     | New York |     9000 |
+---------+----------+----------+
---Depot d1 changes its name to dd1 in Depot------


In [136]:
try:
    cur.execute(query3)
    res = cur.fetchall()
    print(tabulate(res, headers=['prodid', 'deptid', 'quantity'], tablefmt='psql'))
    #q1 = "insert into stock_assign5(prodid,deptid,quantity)values('p3','d1',3000)"
    #q2 = "insert into stock_assign5(prodid,deptid,quantity)values('p2','d1',-400)"
    #q3 = "insert into stock_assign5(prodid,deptid,quantity)values('p1','d1',1000)"
    #cur.execute(q1)
    #cur.execute(q2)
    #cur.execute(q3)
    res = cur.fetchall()
    #print(tabulate(res, headers=['prodid', 'deptid', 'quantity'], tablefmt='psql'))
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Either it is Primary key violation or Please check if there is no row with name p1")
    con.rollback()
finally:
    if con:
        con.commit()


+----------+----------+------------+
| prodid   | deptid   |   quantity |
|----------+----------+------------|
| p3       | d4       |       2000 |
| p2       | d4       |       1500 |
| p3       | d1       |       3000 |
| p2       | d2       |       2000 |
| p2       | d1       |       -400 |
| p1       | d1       |       1000 |
+----------+----------+------------+


In [137]:
#Update Depot d1 changes its name to dd1 in Stock
try:
    query10_1 = "Update stock_assign5 SET deptid = 'dd1' where deptid='d1'"
    print('---Depot d1 changes its name to dd1 in Stock------')
    cur.execute(query10_1)
    cur.execute(query3)
    res = cur.fetchall()
    print(tabulate(res, headers=['prodid', 'deptid', 'quantity'], tablefmt='psql'))
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Either it is Primary key violation or Please check if there is no row with name p1")
    con.rollback()
finally:
    if con:
        con.commit()


---Depot d1 changes its name to dd1 in Stock------
+----------+----------+------------+
| prodid   | deptid   |   quantity |
|----------+----------+------------|
| p3       | d4       |       2000 |
| p2       | d4       |       1500 |
| p2       | d2       |       2000 |
| p3       | dd1      |       3000 |
| p2       | dd1      |       -400 |
| p1       | dd1      |       1000 |
+----------+----------+------------+


In [138]:
# Query11 -- Add product (p100,cd,5) in Product and (p100,d2,50) in Stock
try:
    print('-Add product (p100,cd,5) in Product')
    query11 = "insert into productassign5(productid,pname,price)values('p100','cd',5)"
    cur.execute(query11)
    cur.execute(query1)
    res1 = cur.fetchall()
    print(tabulate(res1, headers=['productid', 'pname', 'price'], tablefmt='psql'))
    # Add and (p100,d2,50) in Stock
    query13 = "insert into stock_assign5(prodid,deptid,quantity) values('p100','d2',50)"
    print("Running the Insertion query in Stock table")
    cur.execute(query13)
    cur.execute(query3)
    res = cur.fetchall()
    print(tabulate(res, headers=['productid', 'deptid', 'quantity'], tablefmt='psql'))
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Either it is Primary key violation or Please check if there is no row with name ")
    con.rollback()
finally:
    if con:
        con.commit()

-Add product (p100,cd,5) in Product
+-------------+---------+---------+
| productid   | pname   |   price |
|-------------+---------+---------|
| p2          | tv      |   250   |
| p3          | vcr     |    80   |
| pp1         | tape    |     2.5 |
| p100        | cd      |     5   |
+-------------+---------+---------+
Running the Insertion query in Stock table
+-------------+----------+------------+
| productid   | deptid   |   quantity |
|-------------+----------+------------|
| p3          | d4       |       2000 |
| p2          | d4       |       1500 |
| p2          | d2       |       2000 |
| p3          | dd1      |       3000 |
| p2          | dd1      |       -400 |
| p1          | dd1      |       1000 |
| p100        | d2       |         50 |
+-------------+----------+------------+


In [140]:
# Query 14 - We add a depot (d100, Chicago, 100) in Depot and (p1, d100, 100) in Stock.
try:
    query14 = "insert into depot_assign5(depid,addr,volume) values('d100','Chicago',100)"    
    cur.execute(query14)
    cur.execute(query2)
    res2 = cur.fetchall()
    print(tabulate(res2, headers=['depid', 'addr', 'volume'], tablefmt='psql'))
        
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Either it is Primary key violation or Please check if there is no row with name ")
    delquery = ""
    con.rollback()
finally:
    if con:
        con.commit()


+---------+----------+----------+
| depid   | addr     |   volume |
|---------+----------+----------|
| d2      | Syracuse |     6000 |
| d4      | New York |     2000 |
| dd1     | New York |     9000 |
| d100    | Chicago  |      100 |
+---------+----------+----------+


In [142]:
query15 = "insert into stock_assign5(prodid,deptid,quantity) values('p100','d100',100)"
try:
    cur.execute(query15)
    cur.execute(query3)
    res3 = cur.fetchall()
  # Print rows in Stock
    print('Added all rows into Stock')
    print(tabulate(res3, headers=['prodid', 'deptid', 'quantity'], tablefmt='psql'))
except(Exception, psycopg2.DatabaseError) as err:
    print(err)
    print("Either it is Primary key violation or Please check if there is no row with name ")
    con.rollback()
finally:
    if con:
        con.commit()

Added all rows into Stock
+----------+----------+------------+
| prodid   | deptid   |   quantity |
|----------+----------+------------|
| p3       | d4       |       2000 |
| p2       | d4       |       1500 |
| p2       | d2       |       2000 |
| p3       | dd1      |       3000 |
| p2       | dd1      |       -400 |
| p1       | dd1      |       1000 |
| p100     | d2       |         50 |
| p100     | d100     |        100 |
+----------+----------+------------+


In [143]:
cur = con.cursor()
query1 = 'Select * from productassign5'
query2 = 'Select * from depot_assign5'
query3 = 'Select * from stock_assign5'

xecuteproject(query1, 'Stock')
xecuteproject(query2, 'Product')
xecuteproject(query3, 'Depot') 

--------------STOCK TABLE-------------------
+---------+--------+----------+
| depid   | addr   |   volume |
|---------+--------+----------|
| p2      | tv     |    250   |
| p3      | vcr    |     80   |
| pp1     | tape   |      2.5 |
| p100    | cd     |      5   |
+---------+--------+----------+
--------------PRODUCT TABLE-------------------
+-------------+----------+------------+
| productid   | deptid   |   quantity |
|-------------+----------+------------|
| d2          | Syracuse |       6000 |
| d4          | New York |       2000 |
| dd1         | New York |       9000 |
| d100        | Chicago  |        100 |
+-------------+----------+------------+
--------------DEPOT TABLE-------------------
+-------------+----------+------------+
| productid   | deptid   |   quantity |
|-------------+----------+------------|
| p3          | d4       |       2000 |
| p2          | d4       |       1500 |
| p2          | d2       |       2000 |
| p3          | dd1      |       3000 |
| p2   